# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 11 2022 2:00PM,248375,19,MSP214562,"Methapharm, Inc.",Released
1,Oct 11 2022 2:00PM,248375,19,MSP214697,"Methapharm, Inc.",Released
2,Oct 11 2022 2:00PM,248375,19,MSP214755,"Methapharm, Inc.",Released
3,Oct 11 2022 2:00PM,248375,19,MSP214759,"Methapharm, Inc.",Released
4,Oct 11 2022 1:23PM,248374,10,8632746,MedStone Pharma LLC,Released
5,Oct 11 2022 1:23PM,248374,10,8632747,MedStone Pharma LLC,Released
6,Oct 11 2022 1:23PM,248374,10,8632748,MedStone Pharma LLC,Released
7,Oct 11 2022 1:23PM,248374,10,8632749,MedStone Pharma LLC,Released
8,Oct 11 2022 1:23PM,248374,10,8632751,MedStone Pharma LLC,Released
9,Oct 11 2022 1:23PM,248374,10,8632752,MedStone Pharma LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,248369,Executing,5
26,248372,Executing,2
27,248372,Released,10
28,248374,Released,6
29,248375,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248366,NaN,NaN,1.0
248369,NaN,5.0,NaN
248372,NaN,2.0,10.0
248374,NaN,NaN,6.0
248375,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248034,0.0,0.0,1.0
248036,0.0,0.0,1.0
248038,0.0,0.0,1.0
248080,0.0,0.0,1.0
248264,0.0,21.0,9.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248034,0,0,1
248036,0,0,1
248038,0,0,1
248080,0,0,1
248264,0,21,9


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248034,0,0,1
1,248036,0,0,1
2,248038,0,0,1
3,248080,0,0,1
4,248264,0,21,9


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248034,,,1
1,248036,,,1
2,248038,,,1
3,248080,,,1
4,248264,,21,9


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 11 2022 2:00PM,248375,19,"Methapharm, Inc."
4,Oct 11 2022 1:23PM,248374,10,MedStone Pharma LLC
10,Oct 11 2022 1:20PM,248372,10,"Senseonics, Incorporated"
22,Oct 11 2022 1:10PM,248369,10,Methapharm-G
25,Oct 11 2022 1:10PM,248369,10,"Methapharm, Inc."
27,Oct 11 2022 1:03PM,248038,21,"NBTY Global, Inc."
28,Oct 11 2022 1:00PM,248080,21,"NBTY Global, Inc."
29,Oct 11 2022 12:58PM,248036,21,"NBTY Global, Inc."
30,Oct 11 2022 12:58PM,248366,19,ACG North America LLC
31,Oct 11 2022 12:46PM,248363,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 11 2022 2:00PM,248375,19,"Methapharm, Inc.",,,4
1,Oct 11 2022 1:23PM,248374,10,MedStone Pharma LLC,,,6
2,Oct 11 2022 1:20PM,248372,10,"Senseonics, Incorporated",,2,10
3,Oct 11 2022 1:10PM,248369,10,Methapharm-G,,5,
4,Oct 11 2022 1:10PM,248369,10,"Methapharm, Inc.",,5,
5,Oct 11 2022 1:03PM,248038,21,"NBTY Global, Inc.",,,1
6,Oct 11 2022 1:00PM,248080,21,"NBTY Global, Inc.",,,1
7,Oct 11 2022 12:58PM,248036,21,"NBTY Global, Inc.",,,1
8,Oct 11 2022 12:58PM,248366,19,ACG North America LLC,,,1
9,Oct 11 2022 12:46PM,248363,19,"AdvaGen Pharma, Ltd",,15,16


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 11 2022 2:00PM,248375,19,"Methapharm, Inc.",4,,
1,Oct 11 2022 1:23PM,248374,10,MedStone Pharma LLC,6,,
2,Oct 11 2022 1:20PM,248372,10,"Senseonics, Incorporated",10,2,
3,Oct 11 2022 1:10PM,248369,10,Methapharm-G,,5,
4,Oct 11 2022 1:10PM,248369,10,"Methapharm, Inc.",,5,
5,Oct 11 2022 1:03PM,248038,21,"NBTY Global, Inc.",1,,
6,Oct 11 2022 1:00PM,248080,21,"NBTY Global, Inc.",1,,
7,Oct 11 2022 12:58PM,248036,21,"NBTY Global, Inc.",1,,
8,Oct 11 2022 12:58PM,248366,19,ACG North America LLC,1,,
9,Oct 11 2022 12:46PM,248363,19,"AdvaGen Pharma, Ltd",16,15,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 11 2022 2:00PM,248375,19,"Methapharm, Inc.",4,,
1,Oct 11 2022 1:23PM,248374,10,MedStone Pharma LLC,6,,
2,Oct 11 2022 1:20PM,248372,10,"Senseonics, Incorporated",10,2,
3,Oct 11 2022 1:10PM,248369,10,Methapharm-G,,5,
4,Oct 11 2022 1:10PM,248369,10,"Methapharm, Inc.",,5,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 11 2022 2:00PM,248375,19,"Methapharm, Inc.",4.0,NaN,NaN
1,Oct 11 2022 1:23PM,248374,10,MedStone Pharma LLC,6.0,NaN,NaN
2,Oct 11 2022 1:20PM,248372,10,"Senseonics, Incorporated",10.0,2.0,NaN
3,Oct 11 2022 1:10PM,248369,10,Methapharm-G,NaN,5.0,NaN
4,Oct 11 2022 1:10PM,248369,10,"Methapharm, Inc.",NaN,5.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 11 2022 2:00PM,248375,19,"Methapharm, Inc.",4.0,0.0,0.0
1,Oct 11 2022 1:23PM,248374,10,MedStone Pharma LLC,6.0,0.0,0.0
2,Oct 11 2022 1:20PM,248372,10,"Senseonics, Incorporated",10.0,2.0,0.0
3,Oct 11 2022 1:10PM,248369,10,Methapharm-G,0.0,5.0,0.0
4,Oct 11 2022 1:10PM,248369,10,"Methapharm, Inc.",0.0,5.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1490196,20.0,13.0,0.0
15,993261,21.0,46.0,0.0
16,993362,4.0,0.0,0.0
18,248317,1.0,0.0,0.0
19,993454,22.0,15.0,0.0
20,248332,1.0,26.0,5.0
21,992188,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1490196,20.0,13.0,0.0
1,15,993261,21.0,46.0,0.0
2,16,993362,4.0,0.0,0.0
3,18,248317,1.0,0.0,0.0
4,19,993454,22.0,15.0,0.0
5,20,248332,1.0,26.0,5.0
6,21,992188,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,20.0,13.0,0.0
1,15,21.0,46.0,0.0
2,16,4.0,0.0,0.0
3,18,1.0,0.0,0.0
4,19,22.0,15.0,0.0
5,20,1.0,26.0,5.0
6,21,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,20.0
1,15,Released,21.0
2,16,Released,4.0
3,18,Released,1.0
4,19,Released,22.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,5.0,0.0
Executing,13.0,46.0,0.0,0.0,15.0,26.0,0.0
Released,20.0,21.0,4.0,1.0,22.0,1.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,5.0,0.0
1,Executing,13.0,46.0,0.0,0.0,15.0,26.0,0.0
2,Released,20.0,21.0,4.0,1.0,22.0,1.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,5.0,0.0
1,Executing,13.0,46.0,0.0,0.0,15.0,26.0,0.0
2,Released,20.0,21.0,4.0,1.0,22.0,1.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()